In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as f
import spacy
import sklearn as sl

In [59]:
import re
import string

In [3]:
# Import the prebuilt word embeddings, upon which the incoming dataset will be convolved.
word_embeddings = spacy.load('en', vectors='glove.6B.300d.txt')

In [4]:
def sequence_to_data(seq, max_len=None):    ####Converting sequence to data basically convertig words to vectors
    seq = unicode(seq, 'utf-8')
    data = [word_embeddings(ix).vector for ix in seq.split()]
    if max_len == None:
        max_len = len(data)
        
    data_mat = np.zeros((1, max_len, 384))
    for ix in range(min(max_len, len(data))):
        data_mat[:, ix, :] = data[ix]

    return data_mat

def seq_data_matrix(seq_data, max_len=None):  ####Now Concating different sentences and converting to a matrix
    data = np.concatenate([sequence_to_data(ix, max_len) for ix in seq_data], axis=0)
    return data

In [6]:
df = pd.read_csv("datasets/V1.4_Training.csv")#loading dataset

### PART 1 : Go through the dataset, and correct basic mistakes.

In [10]:
df

,663_3,"""Please enable removing language code from the Dev Center ""language history"" For example if you ever selected ""ru"" and ""ru-ru"" laguages and you published this xap to the Store then it causes Tile localization to show the en-us(default) tile localization which is bad.""",1
0,663_4,"""Note: in your .csproj file, there is a Suppor...",0
1,664_1,"""Wich means the new version not fully replaced...",0
2,664_2,"""Some of my users will still receive the old x...",0
3,664_3,"""The store randomly gives the old xap or the n...",0
4,664_4,"""My app has a WP7 version and a WP8 version XA...",0
5,664_5,"""The wp7 xap works only on WP7 and the wp8 xap...",0
6,665_1,"""Sometimes the Store gives the wrong wp7 xap v...",0
7,665_2,"""It should be an option to remove the ""ru"" lan...",1
8,665_3,"""Currently if you ever mistakenly selected a ""...",0
9,665_5,"""): the store will randomly deliver the old/wr...",0


In [32]:
texts = np.concatenate((['"Please enable removing language code from the Dev Center "language history" For example if you ever selected "ru" and "ru-ru" laguages and you published this xap to the Store then it causes Tile localization to show the en-us(default) tile localization which is bad."'],df.iloc[:]['"Please enable removing language code from the Dev Center "language history" For example if you ever selected "ru" and "ru-ru" laguages and you published this xap to the Store then it causes Tile localization to show the en-us(default) tile localization which is bad."']),axis=0)

In [26]:
labels = np.concatenate(([1],df.iloc[:]['1'].values),axis=0)

In [33]:
len(labels),len(texts)

(8500, 8500)

In [34]:
texts = [text.replace('"','') for text in texts]

In [37]:
new_dataframe = pd.DataFrame({'texts':[],'labels':[]})

In [38]:
new_dataframe['texts'] = texts
new_dataframe['labels'] = labels

In [39]:
new_dataframe.head()

,texts,labels
0,Please enable removing language code from the ...,1
1,"Note: in your .csproj file, there is a Support...",0
2,Wich means the new version not fully replaced ...,0
3,Some of my users will still receive the old xa...,0
4,The store randomly gives the old xap or the ne...,0


In [40]:
new_dataframe['len'] = ' '

In [42]:
list(labels).count(1)

2085

In [43]:
list(labels).count(0)

6415

The ratio of classes is 3:1 for classes 0:1, which is neither a good ratio, nor a bad one, however it may pose the problem of class imbalance. So I'll first ignore this problem and proceed, then take care of this problem and proceed thence. I'll then compare the F1 scores of both to see if there's a major boost.

##### So, the dataset had the problem of the first row mismatch, I corrected that and created a new dataframe with texts and their corresponding labels.

## PART 2 : Text preprocessing

1. Lowercase every text
2. Remove all numbers
3. Remove punctuation ([!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]) and urls
4. Remove whitespaces


In [99]:
# Lowercase and remove whitespaces
texts = [text.lower().strip() for text in texts]


# Remove numbers
texts = [re.sub(r'\d+', '', text) for text in texts]


#Remove Punctuation
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
for i in range(len(texts)):
    texts[i] = ' '.join(tokenizer.tokenize(texts[i]))
    

In [101]:
#Now apply the embeddings
new_dataframe['len'] = [len(text) for text in texts]

In [104]:
max(new_dataframe['len'].values)

1768